In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 800)
pd.set_option('display.max_rows', 200)

%matplotlib inline

In [36]:
df = pd.read_csv('Current headcount data.csv')

In [37]:
df.head(n=1)

,Employee ID,Start_Date,Leave_date,Grade,Comp_Ratio,Cont._Hours,Annual_Salary,Hourly_Rate,Actual_monthly_pay,Time_in_latest_role,Commute,Direct_Reports,Is_manager,Team_eNPS_avg_response,Country,Function,Employee_Group,Gender,Talent_Planning
0,1,23/06/2013,31/12/9999,4,0.93,35,14222,8,273,0.6,30.4,0,0,1,Indonesia,Operations,Other,Female,Other


In [38]:
from datetime import date

df['Start_Date'] = pd.to_datetime(df['Start_Date'])

df['Start_Date'] = df['Start_Date'].dt.date

df['Leave_date'] = df['Start_Date'].apply(lambda x: (date(2020, 4, 1) - x).days)

df = df.rename(columns={'Leave_date': 'Days at company'})

# Modeling the data: Method 1

In [39]:
df.iloc[:,2:20]

,Days at company,Grade,Comp_Ratio,Cont._Hours,Annual_Salary,Hourly_Rate,Actual_monthly_pay,Time_in_latest_role,Commute,Direct_Reports,Is_manager,Team_eNPS_avg_response,Country,Function,Employee_Group,Gender,Talent_Planning
0,2474,4,0.93,35,14222,8,273,0.6,30.4,0,0,1,Indonesia,Operations,Other,Female,Other
1,2591,4,1.02,35,15606,9,300,7.7,1.8,4,1,5,Other,Marketing,Other,Male,Other
2,2562,11,0.90,21,26179,14,302,7.8,13.9,4,1,6,Brazil,Sales,Standard,Female,Other
3,1976,6,0.99,35,18224,10,350,2.4,8.5,6,1,10,Other,Operations,Standard,Female,Other
4,2196,7,1.02,35,21066,12,405,2.0,9.6,0,0,10,Germany,Sales,Other,Male,Other
5,979,13,0.91,14,30221,17,232,3.4,8.9,3,1,3,Indonesia,Other,Standard,Male,Other
6,2931,5,0.96,35,15662,9,301,2.3,17.7,0,0,3,Other,Finance,Standard,Male,Other
7,903,5,0.94,35,15416,8,296,3.1,1.1,0,0,6,Russian Fed.,Sales,Standard,Female,Other
8,5623,8,0.82,35,18617,10,358,9.7,35.6,0,0,10,Other,Operations,Standard,Female,Other
9,5855,8,0.99,35,22363,12,430,10.4,19.3,6,1,4,Monrovia,Information Services,Other,Male,Other


In [40]:
data = df.iloc[:,2:20]

In [41]:
# Extract feature (X) columns
feature_cols = list(data.columns) 
print("Feature column(s):-\n{}".format(feature_cols))

X_all = data[feature_cols]
print("\nFeature values:-")
print(X_all.head())

Feature column(s):-
['Days at company', 'Grade', 'Comp_Ratio', 'Cont._Hours', 'Annual_Salary', 'Hourly_Rate', 'Actual_monthly_pay', 'Time_in_latest_role', 'Commute', 'Direct_Reports', 'Is_manager', 'Team_eNPS_avg_response', 'Country', 'Function', 'Employee_Group', 'Gender', 'Talent_Planning']

Feature values:-
   Days at company  Grade  Comp_Ratio  Cont._Hours  Annual_Salary  \
0             2474      4        0.93           35          14222   
1             2591      4        1.02           35          15606   
2             2562     11        0.90           21          26179   
3             1976      6        0.99           35          18224   
4             2196      7        1.02           35          21066   

   Hourly_Rate  Actual_monthly_pay  Time_in_latest_role  Commute  \
0            8                 273                  0.6     30.4   
1            9                 300                  7.7      1.8   
2           14                 302                  7.8     13.9   
3

In [42]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print("Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48):-
['Days at company', 'Grade', 'Comp_Ratio', 'Cont._Hours', 'Annual_Salary', 'Hourly_Rate', 'Actual_monthly_pay', 'Time_in_latest_role', 'Commute', 'Direct_Reports', 'Is_manager', 'Team_eNPS_avg_response', 'Country_Argentina', 'Country_Brazil', 'Country_Country X', 'Country_Germany', 'Country_Indonesia', 'Country_Italy', 'Country_Japan', 'Country_Mexico', 'Country_Monrovia', 'Country_Other', 'Country_Pakistan', 'Country_Philippines', 'Country_Poland', 'Country_Romania', 'Country_Russian Fed.', 'Country_South Korea', 'Country_Switzerland', 'Country_Turkey', 'Country_Ukraine', 'Function_External Affairs', 'Function_Finance', 'Function_Information Services', 'Function_Marketing', 'Function_Operations', 'Function_Other', 'Function_People & Culture', 'Function_Sales', 'Function_Science & Innovation', 'Function_Support Services', 'Employee_Group_Other', 'Employee_Group_Standard', 'Employee_Group_Temporary', 'Gender_Female', 'Gender_Male', 'Talent_Planning_High 

In [43]:
X_all.head(n=1)

,Days at company,Grade,Comp_Ratio,Cont._Hours,Annual_Salary,Hourly_Rate,Actual_monthly_pay,Time_in_latest_role,Commute,Direct_Reports,Is_manager,Team_eNPS_avg_response,Country_Argentina,Country_Brazil,Country_Country X,Country_Germany,Country_Indonesia,Country_Italy,Country_Japan,Country_Mexico,Country_Monrovia,Country_Other,Country_Pakistan,Country_Philippines,Country_Poland,Country_Romania,Country_Russian Fed.,Country_South Korea,Country_Switzerland,Country_Turkey,Country_Ukraine,Function_External Affairs,Function_Finance,Function_Information Services,Function_Marketing,Function_Operations,Function_Other,Function_People & Culture,Function_Sales,Function_Science & Innovation,Function_Support Services,Employee_Group_Other,Employee_Group_Standard,Employee_Group_Temporary,Gender_Female,Gender_Male,Talent_Planning_High Potentials,Talent_Planning_Other
0,2474,4,0.93,35,14222,8,273,0.6,30.4,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1


In [44]:
import joblib
clf = joblib.load('model_attrition.pkl')

In [45]:
y_pred = clf.predict(X_all)

In [46]:
y_pred_proba = clf.predict_proba(X_all)

In [47]:
y_pred_proba[0][1]

0.0076567768774478346

In [48]:
y_pred_proba[:, 1]

array([7.65677688e-03, 1.28155612e-02, 7.77249858e-04, ...,
       9.16412924e-01, 9.52322541e-01, 9.89287682e-01])

In [49]:
df['Leaver'] = y_pred
df['Probability of leaving'] = y_pred_proba[:, 1]

In [50]:
df.head()

,Employee ID,Start_Date,Days at company,Grade,Comp_Ratio,Cont._Hours,Annual_Salary,Hourly_Rate,Actual_monthly_pay,Time_in_latest_role,Commute,Direct_Reports,Is_manager,Team_eNPS_avg_response,Country,Function,Employee_Group,Gender,Talent_Planning,Leaver,Probability of leaving
0,1,2013-06-23,2474,4,0.93,35,14222,8,273,0.6,30.4,0,0,1,Indonesia,Operations,Other,Female,Other,0,0.007657
1,2,2013-02-26,2591,4,1.02,35,15606,9,300,7.7,1.8,4,1,5,Other,Marketing,Other,Male,Other,0,0.012816
2,3,2013-03-27,2562,11,0.90,21,26179,14,302,7.8,13.9,4,1,6,Brazil,Sales,Standard,Female,Other,0,0.000777
3,4,2014-11-03,1976,6,0.99,35,18224,10,350,2.4,8.5,6,1,10,Other,Operations,Standard,Female,Other,0,0.085508
4,5,2014-03-28,2196,7,1.02,35,21066,12,405,2.0,9.6,0,0,10,Germany,Sales,Other,Male,Other,0,0.044621


In [51]:
df.loc[df['Leaver']==1].head()

,Employee ID,Start_Date,Days at company,Grade,Comp_Ratio,Cont._Hours,Annual_Salary,Hourly_Rate,Actual_monthly_pay,Time_in_latest_role,Commute,Direct_Reports,Is_manager,Team_eNPS_avg_response,Country,Function,Employee_Group,Gender,Talent_Planning,Leaver,Probability of leaving
73,83,2017-06-05,1031,8,0.82,35,18617,10,358,2.5,9.3,0,0,3,Italy,Operations,Standard,Male,Other,1,0.846615
223,259,2017-12-04,849,10,1.02,28,27512,15,423,2.5,20.3,3,1,4,Turkey,Marketing,Standard,Male,Other,1,0.871228
250,291,2017-07-08,998,16,0.82,35,32401,18,623,3.4,20.8,0,0,6,Other,Operations,Standard,Male,Other,1,0.697402
317,364,2016-04-25,1437,3,0.96,35,10278,6,198,4.7,15.6,0,0,5,Philippines,Operations,Standard,Male,Other,1,0.617897
324,371,2018-12-24,464,2,1.05,35,10176,6,196,1.0,28.4,1,1,9,Russian Fed.,Operations,Standard,Male,Other,1,0.722131


In [52]:
df.to_csv('attrition_modelling.csv', index=False)

In [2]:
df = pd.read_csv('attrition_modelling.csv')

In [4]:
df.head()

,Employee ID,Start_Date,Days at company,Grade,Comp_Ratio,Cont._Hours,Annual_Salary,Hourly_Rate,Actual_monthly_pay,Time_in_latest_role,Commute,Direct_Reports,Is_manager,Team_eNPS_avg_response,Country,Function,Employee_Group,Gender,Talent_Planning,Leaver,Probability of leaving
0,1,2013-06-23,2474,4,0.93,35,14222,8,273,0.6,30.4,0,0,1,Indonesia,Operations,Other,Female,Other,0,0.007657
1,2,2013-02-26,2591,4,1.02,35,15606,9,300,7.7,1.8,4,1,5,Other,Marketing,Other,Male,Other,0,0.012816
2,3,2013-03-27,2562,11,0.90,21,26179,14,302,7.8,13.9,4,1,6,Brazil,Sales,Standard,Female,Other,0,0.000777
3,4,2014-11-03,1976,6,0.99,35,18224,10,350,2.4,8.5,6,1,10,Other,Operations,Standard,Female,Other,0,0.085508
4,5,2014-03-28,2196,7,1.02,35,21066,12,405,2.0,9.6,0,0,10,Germany,Sales,Other,Male,Other,0,0.044621


In [9]:
df.groupby(['Country']).sum()

,Employee ID,Days at company,Grade,Comp_Ratio,Cont._Hours,Annual_Salary,Hourly_Rate,Actual_monthly_pay,Time_in_latest_role,Commute,Direct_Reports,Is_manager,Team_eNPS_avg_response,Leaver,Probability of leaving
Country,,,,,,,,,,,,,,,
Argentina,10931230,2921230,8152,894.38,28077,22417746,12318,390722,4430.6,16005.7,1447,371,5355,42,52.823212
Brazil,10351622,2907094,8309,888.69,28147,22666181,12439,394205,4492.0,16377.1,1524,382,5398,35,42.512834
Country X,3243575,758748,2170,241.58,7490,6007226,3296,103765,1238.2,4577.4,393,97,1430,35,37.014583
Germany,10814126,2134664,6175,678.41,20755,17126490,9401,291778,3156.8,12449.2,1087,280,4147,162,169.767068
Indonesia,76995439,20099577,56454,6126.53,190547,155136914,85251,2662871,30851.0,112247.8,9194,2291,37164,392,472.122082
Italy,10267489,2156007,6189,677.96,20846,17161524,9422,294110,3215.2,12295.6,1160,278,4215,140,148.265318
Japan,11256788,2134981,6313,675.34,21182,17325527,9526,301780,3182.6,12261.7,1020,266,4158,173,186.999657
Mexico,7181114,1423017,4325,454.41,14399,11749051,6452,205672,2119.6,8086.0,671,174,2745,106,110.207422
Monrovia,5627413,1361524,3894,426.35,13398,10740947,5908,186428,2145.9,7618.0,644,155,2565,52,59.092855
